In [1]:
import sys
sys.path.append('../../')

from MPRA_predict.utils import *
import tensorflow as tf

/home/hxcai/anaconda3/envs/torch/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-12-09 17:22:46.654594: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-09 17:22:46.661025: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733736166.668638    8944 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733736166.670927    8944 cuda_blas.cc:141

In [2]:
import glob
import json
import functools


def organism_path(organism):
  return os.path.join('/home/shared/enformer_data', organism)


def get_dataset(organism, subset, num_threads=8):
  metadata = get_metadata(organism)
  dataset = tf.data.TFRecordDataset(tfrecord_files(organism, subset),
                                    compression_type='ZLIB',
                                    num_parallel_reads=num_threads)
  dataset = dataset.map(functools.partial(deserialize, metadata=metadata),
                        num_parallel_calls=num_threads)
  return dataset


def get_metadata(organism):
  # Keys:
  # num_targets, train_seqs, valid_seqs, test_seqs, seq_length,
  # pool_width, crop_bp, target_length
  path = os.path.join(organism_path(organism), 'statistics.json')
  with tf.io.gfile.GFile(path, 'r') as f:
    return json.load(f)


def tfrecord_files(organism, subset):
  # Sort the values by int(*).
  return sorted(tf.io.gfile.glob(os.path.join(
      organism_path(organism), 'tfrecords', f'{subset}-*.tfr'
  )), key=lambda x: int(x.split('-')[-1].split('.')[0]))


def deserialize(serialized_example, metadata):
  """Deserialize bytes stored in TFRecordFile."""
  feature_map = {
      'sequence': tf.io.FixedLenFeature([], tf.string),
      'target': tf.io.FixedLenFeature([], tf.string),
  }
  example = tf.io.parse_example(serialized_example, feature_map)
  sequence = tf.io.decode_raw(example['sequence'], tf.bool)
  sequence = tf.reshape(sequence, (metadata['seq_length'], 4))
  sequence = tf.cast(sequence, tf.float32)

  target = tf.io.decode_raw(example['target'], tf.float16)
  target = tf.reshape(target,
                      (metadata['target_length'], metadata['num_targets']))
  target = tf.cast(target, tf.float32)

  return {'sequence': sequence,
          'target': target}

In [3]:
human_dataset = get_dataset('human', 'train').batch(1).repeat()

2024-12-09 16:39:38.402043: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-12-09 16:39:38.402054: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: wanglab-server
2024-12-09 16:39:38.402057: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: wanglab-server
2024-12-09 16:39:38.402087: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 535.183.1
2024-12-09 16:39:38.402094: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 535.183.1
2024-12-09 16:39:38.402095: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 535.183.1
2024-12-09 16:39:38.402218: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimiz

In [4]:
it = iter(human_dataset)
example = next(it)
print({k: (v.shape, v.dtype) for k,v in example.items()})

{'sequence': (TensorShape([1, 131072, 4]), tf.float32), 'target': (TensorShape([1, 896, 5313]), tf.float32)}


In [5]:
def get_metadata(organism):
    # Keys:
    # num_targets, train_seqs, valid_seqs, test_seqs, seq_length,
    # pool_width, crop_bp, target_length
    path = os.path.join(organism_path(organism), 'statistics.json')
    with tf.io.gfile.GFile(path, 'r') as f:
        metadata = json.load(f)
    return metadata

metadata = get_metadata('human')

dataset = tf.data.TFRecordDataset(tfrecord_files('human', 'train'),
                                  compression_type='ZLIB',
                                  num_parallel_reads=8)
dataset = dataset.map(functools.partial(deserialize, metadata=metadata), 
                      num_parallel_calls=8)

In [6]:
metadata['train_seqs']

34021

In [12]:
it = iter(dataset)
example = next(it)
print({k: (v.shape, v.dtype) for k,v in example.items()})

{'sequence': (TensorShape([131072, 4]), tf.float32), 'target': (TensorShape([896, 5313]), tf.float32)}


In [14]:
def evaluate_model(model, dataset, head, max_steps=None):
    @tf.function
    def predict(x):
        return model(x, is_training=False)[head]
    
    pred = []
    for i, batch in tqdm(enumerate(dataset)):
        if max_steps is not None and i > max_steps:
            break
        pred.append(predict(batch['sequence']))
    return pred

In [7]:
import sys
sys.path.append("../../")

from torch.utils.data import DataLoader
from enformer_pytorch import Enformer, from_pretrained

from MPRA_predict.utils import *
from MPRA_predict.datasets import *

In [ ]:
model = 

In [ ]:
evaluate_model()